In [1]:
import json
from glob import glob

In [2]:
files = sorted(glob('*-dictionary.jsonl'))
files

['negeri-johor-dictionary.jsonl',
 'negeri-kedah-dictionary.jsonl',
 'negeri-kelantan-dictionary.jsonl',
 'negeri-melaka-dictionary.jsonl',
 'negeri-pahang-dictionary.jsonl',
 'negeri-perak-dictionary.jsonl',
 'negeri-sabah-dictionary.jsonl',
 'negeri-sarawak-dictionary.jsonl',
 'negeri-sembilan-dictionary.jsonl',
 'negeri-terengganu-dictionary.jsonl']

In [4]:
templates1 = [
    'apa mksud {perkataan} untuk dialek {bahasa}',
    'mksud {perkataan} dialek {bahasa}',
    'for dialect {bahasa}, meaning {perkataan}',
    'what is the meaning for {perkataan} for dialect {bahasa}',
]

templates2 = [
"""
dalam dialek {bahasa}, 

perkataan: {perkataan}

boleh terangkan dan boleh bagi contoh ayat tak

bagi dalam format kamus:

contoh ayat:
maksud:
penjelasan:
""",
"""
dalam dialek {bahasa}, 

word: {perkataan}

bagi dalam format kamus:

maksud:
penjelasan:
contoh ayat:
""",
"""
dalam dialek {bahasa}, 

{perkataan}

bagi dalam format kamus:

maksud:
penjelasan:
"""
]

In [5]:
import random

instructions = []
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            
            if len(l['dictionary']['maksud']) <= 3:
                continue
            
            if len(l['dictionary']['penjelasan']) <= 3:
                continue
                
            bahasa = f.split('/')[-1].split('-dictionary')[0].replace('-', ' ')
            perkataan = l['left'].lower()
            i = random.choice(templates1).format(perkataan = perkataan, bahasa = bahasa).strip()
            a = f"{l['dictionary']['maksud']}. {l['dictionary']['penjelasan']}"
            instructions.append({
                'prompt_input': None,
                'input': i,
                'output': a,
            })
            
            index = random.randint(0, len(templates2) - 1)
            if index == 0:
                a = "contoh ayat: {contoh_ayat}\nmaksud: {maksud}\npenjelasan: {penjelasan}"
            elif index == 1:
                a = "maksud: {maksud}\npenjelasan: {penjelasan}\ncontoh ayat: {contoh_ayat}"
            else:
                a = "maksud: {maksud}\npenjelasan: {penjelasan}"
            
            i = templates2[index].format(perkataan = perkataan, bahasa = bahasa).strip()
            a = a.format(maksud = l['dictionary']['maksud'], 
                         penjelasan = l['dictionary']['penjelasan'],
                        contoh_ayat = l['dictionary']['contoh_ayat'])
            
            instructions.append({
                'prompt_input': None,
                'input': i,
                'output': a,
            })
            
len(instructions)

13514

In [7]:
instructions[1]

{'prompt_input': None,
 'input': 'dalam dialek negeri johor, \n\nsememeh\n\nbagi dalam format kamus:\n\nmaksud:\npenjelasan:',
 'output': 'maksud: Comot, kotor, tidak bersih\npenjelasan: Perkataan "sememeh" dalam dialek Johor merujuk kepada keadaan sesuatu yang kotor, comot, atau tidak bersih. Ia biasanya digunakan untuk menggambarkan penampilan seseorang atau keadaan sesuatu objek yang kelihatan tidak terurus atau kotor.'}

In [8]:
from collections import defaultdict

languages = defaultdict(dict)
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            bahasa = f.split('/')[-1].split('-dictionary')[0].replace('-', ' ')
            perkataan = l['left'].lower()
            i = perkataan
            if l['dictionary']['maksud'][-1] == '.':
                maksud = l['dictionary']['maksud'][:-1]
            else:
                maksud = l['dictionary']['maksud']
            
            a = f"{maksud}. {l['dictionary']['penjelasan']}"
            languages[bahasa][i] = a

In [13]:
sublanguages = list(languages.keys())

In [14]:
templates_q = [
    'apa mksud perkataan "{perkataan}" dalam dialek {bahasa}',
    'what is the meaning "{perkataan}" in dialect {bahasa}',
    'dlm dialek {bahasa}, apa maksud perkataan {perkataan}'
]

templates_a = [
    '"{perkataan}" bukanlah satu perkataan dari dialek {bahasa}, ia dari dialek {bahasa_2} yang bermaksud {maksud}.',
    "Ia bermaksud {maksud}, tetapi bukanlah satu perkataan dari dialek {bahasa} tetapi dari dialek {bahasa_2}.",
]

In [18]:
instructions_negate = []

for k, v in languages.items():
    if 'sembilan' in k:
        k_ = k.title()
    else:
        k_ = k.replace('negeri ', '').title()
    for word in v:
        
        subs = random.sample(list(set(sublanguages) - set([k])), 5)
        count = 0
        for s in subs:
            if 'sembilan' in s:
                s_ = s.title()
            else:
                s_ = s.replace('negeri ', '').title()
            if word not in languages[s]:
                maksud = v[word]
                if maksud[-1] == '.':
                    maksud = maksud[:-1]
                if maksud[0].isupper():
                    maksud = maksud[0].lower() + maksud[1:]
                i = random.choice(templates_q).strip().format(perkataan = word, bahasa = s_, bahasa_2 = k_)
                a = random.choice(templates_a).strip().format(perkataan = word, bahasa = s_, maksud = maksud, bahasa_2 = k_)
                instructions_negate.append({
                    'prompt_input': None,
                    'input': i,
                    'output': a,
                })

                count += 1

            if count >= 5:
                break
            
len(instructions_negate)

29759

In [19]:
instructions_negate[-1]

{'prompt_input': None,
 'input': 'what is the meaning "jula" in dialect Pahang',
 'output': '"jula" bukanlah satu perkataan dari dialek Pahang, ia dari dialek Terengganu yang bermaksud dahan (cabang pokok). Dalam dialek Terengganu, "jula" merujuk kepada dahan atau cabang pokok. Ia adalah bahagian kayu yang tumbuh dari batang utama pokok dan biasanya mempunyai daun-daun yang tumbuh padanya.'}

In [20]:
templates_q = [
    'apa mksud perkataan "{perkataan}" dalam dialek {bahasa}',
    'what is the meaning "{perkataan}" in dialect {bahasa}',
    'dlm dialek {bahasa}, apa maksud perkataan {perkataan}'
]

templates_a = [
    '"{perkataan}" bermaksud {maksud}',
    'Ia bermaksud {maksud}',
]

In [21]:
instructions_pos = []

for k, v in languages.items():
    if 'sembilan' in k:
        k_ = k.title()
    else:
        k_ = k.replace('negeri ', '').title()
        
    for word in v:
        maksud = v[word]
        
        if maksud[-1] == '.':
            maksud = maksud[:-1]
        if maksud[0].isupper():
            maksud = maksud[0].lower() + maksud[1:]
        i = random.choice(templates_q).strip().format(perkataan = word, bahasa = k_)
        a = random.choice(templates_a).strip().format(perkataan = word, maksud = maksud)
        instructions_pos.append({
            'prompt_input': None,
            'input': i,
            'output': a,
        })
            
len(instructions_pos)

6040

In [22]:
instructions_pos[0]

{'prompt_input': None,
 'input': 'dlm dialek Johor, apa maksud perkataan sememeh',
 'output': '"sememeh" bermaksud comot, kotor, tidak bersih. Perkataan "sememeh" dalam dialek Johor merujuk kepada keadaan sesuatu yang kotor, comot, atau tidak bersih. Ia biasanya digunakan untuk menggambarkan penampilan seseorang atau keadaan sesuatu objek yang kelihatan tidak terurus atau kotor'}

In [23]:
import pandas as pd

df = pd.DataFrame(instructions + instructions_negate + instructions_pos)
df.head()

,prompt_input,input,output
0,None,apa mksud sememeh untuk dialek negeri johor,"Comot, kotor, tidak bersih. Perkataan ""sememeh..."
1,None,"dalam dialek negeri johor, \n\nsememeh\n\nbagi...","maksud: Comot, kotor, tidak bersih\npenjelasan..."
2,None,what is the meaning for memadah for dialect ne...,"Memberitahu, memaklumkan, atau menyampaikan se..."
3,None,"dalam dialek negeri johor, \n\nword: memadah\n...","maksud: Memberitahu, memaklumkan, atau menyamp..."
4,None,what is the meaning for lokos for dialect nege...,"basah. Dalam dialek Johor, perkataan ""lokos"" d..."


In [24]:
df.to_parquet('prepared-malay-dialect-dictionary-instructions.parquet')

In [25]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-malay-dialect-dictionary-instructions.parquet",
    path_in_repo='data/malay_dialect_dictionary_instructions-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-malay-dialect-dictionary-instructions.parquet:   0%|          | 0.00/5.19M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/337aed5cb418b41d0782f07dfdc9e3b8e11271c5', commit_message='Upload data/malay_dialect_dictionary_instructions-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='337aed5cb418b41d0782f07dfdc9e3b8e11271c5', pr_url=None, pr_revision=None, pr_num=None)